**PostgreSQL 10 and above IDENTITY columns :**

PostgreSQL 10 and above have a new *IDENTITY* feature that *supersedes the use of SERIAL*. 

In [1]:
from sqlalchemy import Table, Column, MetaData, Integer, Identity, String

metadata = MetaData()

data = Table(
    "data",
    metadata,
    Column(
        'id', Integer, Identity(start=42, cycle=True), primary_key=True
    ),
    Column('data', String)
)

In [ ]:
CREATE TABLE data (
    id INTEGER GENERATED BY DEFAULT AS IDENTITY (START WITH 42 CYCLE),
    data VARCHAR,
    PRIMARY KEY (id)
)

NOTE : Previous versions of SQLAlchemy did not have built-in support for rendering of IDENTITY, and could use the following compilation hook to replace occurrences of SERIAL with IDENTITY: 

In [ ]:
from sqlalchemy.schema import CreateColumn
from sqlalchemy.ext.compiler import compiles


@compiles(CreateColumn, 'postgresql')
def use_identity(element, compiler, **kw):
    text = compiler.visit_create_column(element, **kw)
    text = text.replace(
        "SERIAL", "INT GENERATED BY DEFAULT AS IDENTITY"
     )
    return text

In [ ]:
# ME : تلاش برای مشاهده نوع های مختف دنباله 

from sqlalchemy import Table, Column, MetaData, Integer, Identity, String, create_engine, insert, URL


url = URL.create(
    drivername="postgresql",
    username="postgres",
    password="1234",
    host="192.168.0.8",
    port="5432",
    database="tests"
)

engine = create_engine(url)
metadata = MetaData()

data = Table(
    "data",
    metadata,
    Column(
        'id', Integer, Identity(start=42, cycle=True), primary_key=True
    ),
    # cycle=True, maxvalue=45 :UniqueViolation: duplicate key value violates unique constraint "data_pkey"
    Column('data', String)
)

data.create(engine)

with engine.connect() as conn :
    conn.execute(
            insert(data),
            [{"data" : i} for i in range(100)]
        )

    #conn.commit()